# Chapter 14: Least Squares

- content: pp. 395 - 414
- exercises: pp. 415 - 419

In [2]:
# import commonly used Python libraries
import numpy as np
from matplotlib import pyplot as plt

## 14.1 Introduction

- The physical and biological world we inhabit is really, really, *really* complicated.
- How do we make sense of the complexity?
- We make simplified models of the most important aspects of the system under investigation, while ignoring or abstracting away the aspects that are less relevant.
- This process leads to a *model*, which is a set of equations that allows scientists to isolate and understand the principles of the system under investigation.
- Thus, the goal of building models is not to replicate *exactly* the system under investigation, but instead to identify a simplified and low-dimensional representation that can be understood by humans or simulated by computers.
- On the other hand, models should be sufficiently generic that they can be applied to different datasets.  This means that the models must be flexible enough that they can be adjusted to different datasets without having to create a brand new model for each particular dataset.
- This is why models contain both *fixed features* and *free parameters*.
  - The fixed features are components of the model that the scientist imposes, based on scientific evidence, theories, and intuition.
  - Free parameters are varaibles that can be adjusted to allow the model to fit any particular dataset.
- This brings us to the primary goal of model-fitting: find values for these free parameters that make the model match the data as closely as possible.

### Example:

- let's say we wanted to predict someone's height.
- Your hypotehsis is that height is a function of 3 features:
  - sex (male/female)
  - parents height
  - childhood nutrition (1-10 scale)
- *obviously what determines height is much more complicated than this but we are trying to simplify and capture the key factors*

we can now construct our model for height:
$$h = \beta_1s + \beta_2p + \beta_3n + \epsilon$$

Fixed Features:
- $h$ = height
- $s$ = sex
- $p$ = parents' height
- $n$ = childhood nutrition score
- $\epsilon$ = an "error term" / "residual" that captures all the variance in height that the 3 variables don't explain

Free Parameters:
- $\beta$ scalars for each of the factors
- they are also often called "weights" because the larger the scalar, the more influence the fixed feature has on the outcome.


- This leads to the question of how you find the model parameters that make the model match the data as closely as possible.
- Random guessing is a terrible idea
- We need an algorithm that will find the best parameters given the data
- The most commonly used algorithm for fitting models to data is **linear least-squares**, which we will learn in this chapter.

## 14.2 The five steps of model-fitting

## 14.3 Terminology

## 14.4 Least-squares via left inverse

## 14.5 Least-squares via orthogonal projection

## 14.6 Least squares via row-reduction

## 14.7 Model-predicted values and residuals

## 14.8 Least-squares example

## 14.9 - 14.10 Code Challenges